In [21]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

In [22]:
df = pd.read_csv("./datasets/diabetic_data_clean.csv")

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## 1. Feature Engineering

In [64]:
df['gender'] = df.gender.replace({'Female':1, 'Male':0, 'Unknown/Invalid':np.nan})

In [24]:
def get_age_bounds(age):
    bounds = re.findall(r'(\w+)', age)
    return int(bounds[0])

df['age_min'] = df['age'].apply(lambda x : get_age_bounds(x))

In [25]:
diags = []
for c in ["diag_1", "diag_2", "diag_3"]:
    diags += list(df[c].unique())
diags = set([d for d in diags if str(d).isalpha() and d is not np.nan])

map_diag = {d:i for i,d in enumerate(diags)}

for c in ["diag_1", "diag_2", "diag_3"]:
    df[c] = df[c].apply(lambda x: np.nan if x not in map_diag.keys() else x )
    df[c] = df[c].replace(map_diag)
    
for v in map_diag.values():
    df['diag_code_'+str(v)] = 0
    for c in ["diag_1", "diag_2", "diag_3"]:
        df['diag_code_'+str(v)] += (df[c] == v).astype(int)

In [26]:
cols_meds = [
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
    'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
    'tolazamide', 'examide', 'citoglipton', 'insulin',
    'glyburide-metformin', 'glipizide-metformin',
    'glimepiride-pioglitazone', 'metformin-rosiglitazone',
    'metformin-pioglitazone'
]

for c in cols_meds:
    df[c] = df[c].replace({'No':-2, 'Steady':0, 'Up':1, 'Down':-1})

In [27]:
df['readmitted_binary'] = df['readmitted'].replace({'NO':0, '>30':0, '<30':1})

In [93]:
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,diag_code_5,diag_code_6,diag_code_7,diag_code_8,diag_code_9,diag_code_10,diag_code_11,diag_code_12,diag_code_13,readmitted_binary
0,2278392,8222157,Caucasian,1.0,[0-10),NaN,6,25,1,1,...,0,0,0,0,0,0,0,0,0,0
1,149190,55629189,Caucasian,1.0,[10-20),NaN,1,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,64410,86047875,AfricanAmerican,1.0,[20-30),NaN,1,1,7,2,...,0,1,0,0,0,0,0,0,0,0
3,500364,82442376,Caucasian,0.0,[30-40),NaN,1,1,7,2,...,1,0,0,0,0,0,0,0,0,0
4,16680,42519267,Caucasian,0.0,[40-50),NaN,1,1,7,1,...,0,0,0,0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,0.0,[70-80),NaN,1,3,7,3,...,1,0,0,1,0,0,0,0,0,0
101762,443847782,74694222,AfricanAmerican,1.0,[80-90),NaN,1,4,5,5,...,0,0,0,0,0,0,0,0,0,0
101763,443854148,41088789,Caucasian,0.0,[70-80),NaN,1,1,7,1,...,0,0,0,1,0,0,1,0,0,0
101764,443857166,31693671,Caucasian,1.0,[80-90),NaN,2,3,7,10,...,0,0,0,0,2,0,0,0,0,0


In [109]:
patient_nbr = 41088789
df[df.patient_nbr == patient_nbr].head().T.iloc[:-10]

,59611,76404,101763
encounter_id,167593854,230612106,443854148
patient_nbr,41088789,41088789,41088789
race,Caucasian,Caucasian,Caucasian
gender,0.0,0.0,0.0
age,[70-80),[70-80),[70-80)
weight,NaN,NaN,NaN
admission_type_id,1,1,1
discharge_disposition_id,1,1,1
admission_source_id,7,7,7
time_in_hospital,3,1,1


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,diag_code_5,diag_code_6,diag_code_7,diag_code_8,diag_code_9,diag_code_10,diag_code_11,diag_code_12,diag_code_13,readmitted_binary
0,2278392,8222157,Caucasian,1.0,[0-10),NaN,6,25,1,1,...,0,0,0,0,0,0,0,0,0,0


## 2. Modeling

### 2.1. Train / Test Split

In [36]:
SEED = 42
TEST_SIZE = 0.33


In [29]:
cols_drop = [
    "weight", "race", "age", "readmitted",
    "payer_code", "medical_specialty",
    "number_outpatient", "number_inpatient", 
    "max_glu_serum", "A1Cresult", 
    "diag_1", "diag_2", "diag_3"
]

In [65]:
X = df.drop(cols_drop, axis=1)
X.dropna(inplace=True)

y = X.readmitted_binary
X = X.drop('readmitted_binary', axis=1)

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                    test_size=TEST_SIZE, random_state=SEED, stratify=y)

## 2.2. Model Training & Fine-tuning

In [87]:
N_TRIALS = 100
EARLY_STOPPING_ROUNDS = 50

In [88]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from optuna import create_study
from optuna.integration import XGBoostPruningCallback

In [91]:
def objective(
    trial,
    X,
    y,
    random_state=22,
    n_splits=3,
    n_jobs=1,
    early_stopping_rounds=50,
):
    # XGBoost parameters
    params = {
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "binary:logistic",
        "n_estimators": 10000,
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        #"learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        #"colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        #"subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        #"alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        #"lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        #"gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        #"min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs, 
        #'early_stopping_rounds': early_stopping_rounds
    }

    model = XGBClassifier(**params)
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-auc")
    
    kf = KFold(
        n_splits=n_splits, random_state=random_state, shuffle=True)
    
    X_values = X.values
    y_values = y.values
    
    auc = 0
    
    for train_index, test_index in kf.split(X_values):
        X_tain, X_test = X_values[train_index, :], X_values[test_index, :]
        y_train, y_test = y_values[train_index], y_values[test_index]
        model.fit(
            X_tain,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="auc",
            verbose=1,
            callbacks=[pruning_callback],
            early_stopping_rounds=early_stopping_rounds,
        )
        y_pred = model.predict(X_test)
        auc += roc_auc_score(y_test, y_pred)
    
    return auc/n_splits

In [92]:
study = create_study(direction="minimize")
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        y_train,
        random_state=SEED,
        n_splits=N_SPLITS,
        n_jobs=4,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
    ),
    n_trials=N_TRIALS,
    n_jobs=1,
)

# display params
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

[I 2023-01-10 19:06:31,693] A new study created in memory with name: no-name-4693507a-cb33-4099-b62e-8bf0492c8aea


[0]	validation_0-auc:0.61457
[1]	validation_0-auc:0.62072
[2]	validation_0-auc:0.62562
[3]	validation_0-auc:0.62897
[4]	validation_0-auc:0.63007
[5]	validation_0-auc:0.63090
[6]	validation_0-auc:0.63353
[7]	validation_0-auc:0.63548
[8]	validation_0-auc:0.63701
[9]	validation_0-auc:0.63851
[10]	validation_0-auc:0.63702
[11]	validation_0-auc:0.63646
[12]	validation_0-auc:0.63724
[13]	validation_0-auc:0.63918
[14]	validation_0-auc:0.64167
[15]	validation_0-auc:0.64212
[16]	validation_0-auc:0.64081
[17]	validation_0-auc:0.64083
[18]	validation_0-auc:0.64012
[19]	validation_0-auc:0.63988
[20]	validation_0-auc:0.64008
[21]	validation_0-auc:0.64055
[22]	validation_0-auc:0.64082
[23]	validation_0-auc:0.64208
[24]	validation_0-auc:0.64141
[25]	validation_0-auc:0.64159
[26]	validation_0-auc:0.64253
[27]	validation_0-auc:0.64216
[28]	validation_0-auc:0.64171
[29]	validation_0-auc:0.64227
[30]	validation_0-auc:0.64215
[31]	validation_0-auc:0.64149
[32]	validation_0-auc:0.64023
[33]	validation_0-au

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/optuna/trial/_trial.py:462: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/optuna/trial/_trial.py:462: UserWarning: The reported value i

[2]	validation_0-auc:0.62045
[3]	validation_0-auc:0.62235
[4]	validation_0-auc:0.62261
[5]	validation_0-auc:0.62293
[6]	validation_0-auc:0.62235


/usr/local/lib/python3.9/site-packages/optuna/trial/_trial.py:462: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/optuna/trial/_trial.py:462: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/optuna/trial/_trial.py:462: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/optuna/trial/_trial.py:462: UserWarning: The reported value is ignored because this `step` 5 is already reported.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/optuna/trial/_trial.py:462: UserWarning: The reported value is ignored because this `step` 6 is already reported.
  warnings.warn(


...

[0]	validation_0-auc:0.61283


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-01-10 19:27:30,222] Trial 43 pruned. Trial was pruned at iteration 0.


[0]	validation_0-auc:0.60863
[1]	validation_0-auc:0.61518
[2]	validation_0-auc:0.61825
[3]	validation_0-auc:0.62055
[4]	validation_0-auc:0.62234
[5]	validation_0-auc:0.62203
[6]	validation_0-auc:0.62388
[7]	validation_0-auc:0.62665
[8]	validation_0-auc:0.62921
[9]	validation_0-auc:0.63081
[10]	validation_0-auc:0.63214
[11]	validation_0-auc:0.63557


[W 2023-01-10 19:27:30,749] Trial 44 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-92-0d3db101c3c0>", line 3, in <lambda>
    lambda trial: objective(
  File "<ipython-input-91-ccc421e2b08d>", line 42, in objective
    model.fit(
  File "/usr/local/lib/python3.9/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py", line 1490, in fit
    self._Booster = train(
  File "/usr/local/lib/python3.9/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.9/site-packages/xgboost/training.py", line 185, in train
    bst.update(dtrain, i, obj)
  File "/usr/local/lib/python3.9/site-packages/xgboost/core.py", line 1918, in update
    _check_call(_LIB.XGB

KeyboardInterrupt: 